## 상의할 내용
- icd_version이 'no'인 경우를 제외하고 icd_version과 icd_version_diag가 모두 값을 갖는 경우 12개의 컬럼을 제외하고는 두 버전이 동일
- 12개의 인스턴스를 그냥 제거할지 말지(현재 제거 안 함)
<br><br>
- 입원한 환자에 대해서만 수행하는 건 어떨지(ed_stay=0.0인 경우)
<br><br>
- 현재 데이터셋의 크기가 너무 크다고 생각함 -> 데이터 수를 줄이는 게 어떨지&만약 줄인다면 어떤 식으로 줄일 건지

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**대표질병진단은 seq=1인 샘플로 추출된 상태**
<br>각자 전처리 수행하면서 임베딩인 다른 방법으로 대표질병진단 추출하고 결과 비교하면서 모델 수행

In [2]:
adm = pd.read_csv("C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\core\\admissions.csv")
pat = pd.read_csv("C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\core\\patients.csv")
trans = pd.read_csv("C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\core\\transfers.csv")

In [3]:
surgery = pd.read_csv('C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\hosp\\procedures_icd.csv')
d_surgery = pd.read_csv('C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\hosp\\d_icd_procedures.csv')

In [4]:
service = pd.read_csv('C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\hosp\\services.csv')

In [5]:
diag = pd.read_csv('C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\hosp\\diagnoses_icd.csv')
d_diag = pd.read_csv('C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\hosp\\d_icd_diagnoses.csv')

In [6]:
drg = pd.read_csv('C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\hosp\\drgcodes.csv')

In [7]:
d_lab = pd.read_csv('C:\\Users\\songa\\OneDrive\\바탕 화면\\데이터애널리틱스\\MIMIC-IV\\hosp\\d_labitems.csv')

In [8]:
# adm, pat 결합: df
df = pd.merge(adm, pat, on = 'subject_id')
df = df[['hadm_id', 'subject_id', 'admittime','edregtime','gender', 'anchor_age']]
df['ed_stay'] =df['edregtime'].notna().astype(int) # ed 입원 여부 변수 추출
df = df[['hadm_id', 'subject_id', 'admittime','ed_stay','gender', 'anchor_age']]
df = df.sort_values(by = ['subject_id'])

# df, diagnoses 데이터 결합: df2
df2 = pd.merge(df, diag, on = 'hadm_id')
df2 = df2.sort_values(by = ['hadm_id', 'seq_num'])
df2 = df2[['hadm_id', 'subject_id_x', 'admittime','ed_stay','gender', 'anchor_age', 'seq_num', 'icd_code',
           'icd_version']]
df2_seq1 = df2.where(df2['seq_num'] == 1).dropna()
# df2_seq: 대표질병진단으로 seq_num 값이 1인 질병진단만 추출해낸 데이터셋

# 이전 병력 정보 변수 추출: df4
df4 = df2_seq1.copy().sort_values(by=['subject_id_x', 'admittime']).reset_index(drop=True)
prev_diag = df4['icd_code'].copy()
for i in range(len(df4)):
    if i == 0:
        prev_diag[i] = 'none' 
        continue
    if df4['subject_id_x'][i] == df4['subject_id_x'][i-1]:
        prev_diag[i] = df4['icd_code'][i-1]
    elif df4['subject_id_x'][i-1] != df4['subject_id_x'][i]:
        prev_diag[i] = 'none'
df4['prev_diag'] = prev_diag

# df4, surgery(수술 데이터) 결합: df3
df3 = pd.merge(df4, surgery, on='hadm_id', how='left').fillna('no')
df3['icd_code_diag'] = df3['icd_code_x']
df3['icd_version_diag'] = df3['icd_version_x']
df3['seq_num'] = df3['seq_num_y']
df3['icd_code'] = df3['icd_code_y']
df3['icd_version'] = df3['icd_version_y']
df3 = df3[['hadm_id', 'ed_stay','gender', 'anchor_age', 'prev_diag',
           'icd_code_diag', 'icd_version_diag', 'seq_num', 'icd_code', 'icd_version']]
df3 = df3.sort_values(by=['hadm_id', 'seq_num']).reset_index(drop=True)

In [9]:
df3

,hadm_id,ed_stay,gender,anchor_age,prev_diag,icd_code_diag,icd_version_diag,seq_num,icd_code,icd_version
0,20000019.0,1.0,F,76.0,none,0389,9.0,no,no,no
1,20000024.0,1.0,F,84.0,S32059A,D500,10.0,no,no,no
2,20000034.0,0.0,M,72.0,K831,K831,10.0,no,no,no
3,20000041.0,0.0,F,62.0,71536,71536,9.0,1.0,8154,9.0
4,20000055.0,0.0,F,0.0,none,V3001,9.0,no,no,no
...,...,...,...,...,...,...,...,...,...,...
1019215,29999745.0,1.0,M,49.0,30981,29680,9.0,no,no,no
1019216,29999785.0,0.0,M,0.0,none,Z3801,10.0,1.0,0VTTXZZ,10.0
1019217,29999809.0,1.0,M,73.0,7291,41401,9.0,no,no,no
1019218,29999828.0,0.0,M,47.0,none,27801,9.0,1.0,4438,9.0


## 데이터 확인

NULL값 확인(없음)

In [10]:
# 각 열의 null 값 개수 확인
print("각 열의 null 값 개수:")
print(df3.isnull().sum())


각 열의 null 값 개수:
hadm_id             0
ed_stay             0
gender              0
anchor_age          0
prev_diag           0
icd_code_diag       0
icd_version_diag    0
seq_num             0
icd_code            0
icd_version         0
dtype: int64


In [11]:
# 각 열의 도메인 값 확인
for column in df3.columns:
    print(f"열 '{column}'의 도메인 값들:")
    print(df3[column].unique())  # 각 열의 고유 값들
    print()


열 'hadm_id'의 도메인 값들:
[20000019. 20000024. 20000034. ... 29999809. 29999828. 29999928.]

열 'ed_stay'의 도메인 값들:
[1. 0.]

열 'gender'의 도메인 값들:
['F' 'M']

열 'anchor_age'의 도메인 값들:
[76. 84. 72. 62.  0. 91. 32. 79. 18. 71. 69. 52. 47. 75. 57. 59. 34. 42.
 35. 82. 61. 53. 55. 49. 51. 65. 50. 45. 60. 67. 66. 43. 54. 20. 26. 73.
 89. 29. 30. 86. 63. 28. 64. 56. 46. 25. 58. 37. 39. 19. 41. 70. 24. 48.
 85. 36. 21. 22. 33. 88. 81. 68. 23. 77. 78. 38. 80. 83. 40. 87. 44. 27.
 74. 31.]

열 'prev_diag'의 도메인 값들:
['none' 'S32059A' 'K831' ... 'S82872C' 'D569' 'S42391A']

열 'icd_code_diag'의 도메인 값들:
['0389' 'D500' 'K831' ... '8783' 'O624' '78094']

열 'icd_version_diag'의 도메인 값들:
[ 9. 10.]

열 'seq_num'의 도메인 값들:
['no' 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 14.0 15.0
 16.0 17.0 18.0 19.0 20.0 21.0 22.0 23.0 24.0 25.0 26.0 27.0 28.0 29.0
 30.0 31.0 32.0 33.0 34.0 35.0 36.0 37.0 38.0 39.0 40.0 41.0]

열 'icd_code'의 도메인 값들:
['no' '8154' '0KQM0ZZ' ... '0S9F0ZX' '0GSP0ZZ' '01NM4ZZ']

열 'icd_version'의 

In [12]:
# 도메인 값과 빈도수 함께 보기
for column in df3.columns:
    print(f"열 '{column}'의 도메인 값과 빈도수:")
    print(df3[column].value_counts())  # 각 열의 값과 빈도수
    print()  
    

열 'hadm_id'의 도메인 값과 빈도수:
25434637.0    41
24352194.0    40
21108661.0    40
27321750.0    40
23202662.0    40
              ..
23967027.0     1
23966981.0     1
23966978.0     1
23966931.0     1
29999928.0     1
Name: hadm_id, Length: 521110, dtype: int64

열 'ed_stay'의 도메인 값과 빈도수:
1.0    544067
0.0    475153
Name: ed_stay, dtype: int64

열 'gender'의 도메인 값과 빈도수:
M    519145
F    500075
Name: gender, dtype: int64

열 'anchor_age'의 도메인 값과 빈도수:
0.0     83848
91.0    21416
63.0    21229
62.0    21054
64.0    21039
        ...  
88.0     6675
21.0     6085
19.0     4708
89.0     4149
18.0     2606
Name: anchor_age, Length: 74, dtype: int64

열 'prev_diag'의 도메인 값과 빈도수:
none       521440
41401       10342
78650        6708
78659        5759
V5811        5030
            ...  
99772           1
T387X1A         1
O695XX0         1
M86462          1
V7181           1
Name: prev_diag, Length: 10140, dtype: int64

열 'icd_code_diag'의 도메인 값과 빈도수:
V3000     28193
41401     26629
Z3800     17434
V3001    

In [13]:
df3

,hadm_id,ed_stay,gender,anchor_age,prev_diag,icd_code_diag,icd_version_diag,seq_num,icd_code,icd_version
0,20000019.0,1.0,F,76.0,none,0389,9.0,no,no,no
1,20000024.0,1.0,F,84.0,S32059A,D500,10.0,no,no,no
2,20000034.0,0.0,M,72.0,K831,K831,10.0,no,no,no
3,20000041.0,0.0,F,62.0,71536,71536,9.0,1.0,8154,9.0
4,20000055.0,0.0,F,0.0,none,V3001,9.0,no,no,no
...,...,...,...,...,...,...,...,...,...,...
1019215,29999745.0,1.0,M,49.0,30981,29680,9.0,no,no,no
1019216,29999785.0,0.0,M,0.0,none,Z3801,10.0,1.0,0VTTXZZ,10.0
1019217,29999809.0,1.0,M,73.0,7291,41401,9.0,no,no,no
1019218,29999828.0,0.0,M,47.0,none,27801,9.0,1.0,4438,9.0


## 인코딩

**누락된 값 처리**

In [14]:
# 'M'을 0으로, 'F'를 1로 임베딩
df3['gender'] = df3['gender'].map({'M': 0, 'F': 1})

In [15]:
# 'seq_num', 'icd_code', 'icd_version' 열에서 'no' 값을 0으로 변환
df3['seq_num'] = df3['seq_num'].replace('no', 0)
df3['icd_code'] = df3['icd_code'].replace('no', 0)
df3['icd_version'] = df3['icd_version'].replace('no', 0)

In [16]:
# 'prev_diag' 열에서 'none' 값을 0으로 변환
df3['prev_diag'] = df3['prev_diag'].replace('none', 0)


In [17]:
df3

,hadm_id,ed_stay,gender,anchor_age,prev_diag,icd_code_diag,icd_version_diag,seq_num,icd_code,icd_version
0,20000019.0,1.0,1,76.0,0,0389,9.0,0.0,0,0.0
1,20000024.0,1.0,1,84.0,S32059A,D500,10.0,0.0,0,0.0
2,20000034.0,0.0,0,72.0,K831,K831,10.0,0.0,0,0.0
3,20000041.0,0.0,1,62.0,71536,71536,9.0,1.0,8154,9.0
4,20000055.0,0.0,1,0.0,0,V3001,9.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
1019215,29999745.0,1.0,0,49.0,30981,29680,9.0,0.0,0,0.0
1019216,29999785.0,0.0,0,0.0,0,Z3801,10.0,1.0,0VTTXZZ,10.0
1019217,29999809.0,1.0,0,73.0,7291,41401,9.0,0.0,0,0.0
1019218,29999828.0,0.0,0,47.0,0,27801,9.0,1.0,4438,9.0


**prev_diag와 icd_code_diag에 대한 해시 인코딩**

In [18]:
# 해시 인코딩을 위한 함수 정의

def hash_encode(df3, column, num_bins=1000):
    """ 해시 인코딩 함수: 주어진 컬럼에 대해 해시 값을 생성하여 num_bins 개의 범주로 변환 """
    df3[column + '_hashed'] = df3[column].apply(lambda x: hash(x) % num_bins)
    return df3

# prev_diag와 icd_code_diag에 대해 해시 인코딩 적용
df3 = hash_encode(df3, 'prev_diag', num_bins=20000)      #10140
df3 = hash_encode(df3, 'icd_code_diag', num_bins=25000)  #13263
df3 = hash_encode(df3, 'icd_code', num_bins=25000)       #12939
df3

,hadm_id,ed_stay,gender,anchor_age,prev_diag,icd_code_diag,icd_version_diag,seq_num,icd_code,icd_version,prev_diag_hashed,icd_code_diag_hashed,icd_code_hashed
0,20000019.0,1.0,1,76.0,0,0389,9.0,0.0,0,0.0,0,24909,0
1,20000024.0,1.0,1,84.0,S32059A,D500,10.0,0.0,0,0.0,18321,14058,0
2,20000034.0,0.0,0,72.0,K831,K831,10.0,0.0,0,0.0,18057,3057,0
3,20000041.0,0.0,1,62.0,71536,71536,9.0,1.0,8154,9.0,10886,20886,508
4,20000055.0,0.0,1,0.0,0,V3001,9.0,0.0,0,0.0,0,21972,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019215,29999745.0,1.0,0,49.0,30981,29680,9.0,0.0,0,0.0,16346,6379,0
1019216,29999785.0,0.0,0,0.0,0,Z3801,10.0,1.0,0VTTXZZ,10.0,0,24593,18960
1019217,29999809.0,1.0,0,73.0,7291,41401,9.0,0.0,0,0.0,3115,4928,0
1019218,29999828.0,0.0,0,47.0,0,27801,9.0,1.0,4438,9.0,0,4337,20201


In [19]:
# 'prev_diag', 'icd_code_diag', 'icd_code' 컬럼 제거
df3 = df3.drop(columns=['prev_diag', 'icd_code_diag', 'icd_code'])

df3

,hadm_id,ed_stay,gender,anchor_age,icd_version_diag,seq_num,icd_version,prev_diag_hashed,icd_code_diag_hashed,icd_code_hashed
0,20000019.0,1.0,1,76.0,9.0,0.0,0.0,0,24909,0
1,20000024.0,1.0,1,84.0,10.0,0.0,0.0,18321,14058,0
2,20000034.0,0.0,0,72.0,10.0,0.0,0.0,18057,3057,0
3,20000041.0,0.0,1,62.0,9.0,1.0,9.0,10886,20886,508
4,20000055.0,0.0,1,0.0,9.0,0.0,0.0,0,21972,0
...,...,...,...,...,...,...,...,...,...,...
1019215,29999745.0,1.0,0,49.0,9.0,0.0,0.0,16346,6379,0
1019216,29999785.0,0.0,0,0.0,10.0,1.0,10.0,0,24593,18960
1019217,29999809.0,1.0,0,73.0,9.0,0.0,0.0,3115,4928,0
1019218,29999828.0,0.0,0,47.0,9.0,1.0,9.0,0,4337,20201


**특정 컬럼들에 대해 정규화 수행**

In [20]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 또는 정규화 (Normalization)
scaler = MinMaxScaler()
df3[['anchor_age', 'prev_diag_hashed', 'icd_code_diag_hashed', 'icd_code_hashed']] = scaler.fit_transform(df3[['anchor_age', 'prev_diag_hashed', 'icd_code_diag_hashed', 'icd_code_hashed']])

# 결과 확인
df3

,hadm_id,ed_stay,gender,anchor_age,icd_version_diag,seq_num,icd_version,prev_diag_hashed,icd_code_diag_hashed,icd_code_hashed
0,20000019.0,1.0,1,0.835165,9.0,0.0,0.0,0.000000,0.996559,0.000000
1,20000024.0,1.0,1,0.923077,10.0,0.0,0.0,0.916096,0.562380,0.000000
2,20000034.0,0.0,0,0.791209,10.0,0.0,0.0,0.902895,0.122199,0.000000
3,20000041.0,0.0,1,0.681319,9.0,1.0,9.0,0.544327,0.835587,0.020321
4,20000055.0,0.0,1,0.000000,9.0,0.0,0.0,0.000000,0.879041,0.000000
...,...,...,...,...,...,...,...,...,...,...
1019215,29999745.0,1.0,0,0.538462,9.0,0.0,0.0,0.817341,0.255122,0.000000
1019216,29999785.0,0.0,0,0.000000,10.0,1.0,10.0,0.000000,0.983915,0.758430
1019217,29999809.0,1.0,0,0.802198,9.0,0.0,0.0,0.155758,0.197063,0.000000
1019218,29999828.0,0.0,0,0.516484,9.0,1.0,9.0,0.000000,0.173415,0.808072


In [21]:
# 데이터 전처리: seq_num에 따라 정렬
df3 = df3.sort_values(by=['hadm_id', 'seq_num'])

## 모델링

**데이터 수가 약 102만개라 hadm_id를 기준으로 10,000개만 랜덤 추출**

In [22]:
# 'hadm_id' 컬럼에서 고유한 값 10,000개를 랜덤으로 추출
sampled_hadm_ids = df3['hadm_id'].drop_duplicates().sample(n=100000, random_state=42)

# 랜덤으로 선택된 'hadm_id'를 가진 인스턴스들만 필터링
df_rd = df3[df3['hadm_id'].isin(sampled_hadm_ids)]

# 필터링된 데이터프레임 출력
df_rd

,hadm_id,ed_stay,gender,anchor_age,icd_version_diag,seq_num,icd_version,prev_diag_hashed,icd_code_diag_hashed,icd_code_hashed
2,20000034.0,0.0,0,0.791209,10.0,0.0,0.0,0.902895,0.122199,0.000000
6,20000069.0,0.0,1,0.351648,10.0,1.0,10.0,0.000000,0.964589,0.135325
7,20000069.0,0.0,1,0.351648,10.0,2.0,10.0,0.000000,0.964589,0.999840
8,20000094.0,0.0,0,0.868132,10.0,0.0,0.0,0.996450,0.026929,0.000000
46,20000397.0,0.0,0,0.670330,10.0,1.0,10.0,0.000000,0.406690,0.090324
...,...,...,...,...,...,...,...,...,...,...
1019194,29999506.0,0.0,1,0.000000,9.0,0.0,0.0,0.000000,0.806418,0.000000
1019195,29999560.0,1.0,1,0.659341,9.0,1.0,9.0,0.000000,0.913132,0.268011
1019196,29999560.0,1.0,1,0.659341,9.0,2.0,9.0,0.000000,0.913132,0.548382
1019215,29999745.0,1.0,0,0.538462,9.0,0.0,0.0,0.817341,0.255122,0.000000


In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [24]:
# 환자별로 시퀀스를 생성하기 위해 hadm_id별로 그룹화
grouped = df_rd.groupby('hadm_id')

# 시퀀스 데이터 준비
X = []
y = []

# 각 환자별로 시퀀스를 만들고, 그에 해당하는 예측 값을 추가
for _, group in grouped:
    # seq_num이 0보다 큰 경우만 선택 (첫 번째 치료 이후의 상태를 예측)
    if len(group) > 1:
        # 필요한 열만 미리 추출
        group_data = group[['anchor_age', 'gender', 'icd_version_diag', 'prev_diag_hashed', 'icd_code_diag_hashed']].values
        for i in range(1, len(group)):
            # X에 이전 상태들 (시간축에 따라 누적된 데이터를 추가)
            X.append(group_data[:i])  # 이전 상태들 (각 환자별로 누적 시퀀스)
            # y에 현재 상태 (예측 값)
            y.append(group.iloc[i]['icd_code_diag_hashed'])  # 현재 상태 (예측값)

X = np.array(X)
y = np.array(y)

# LSTM의 입력 형식에 맞게 X의 shape을 (samples, time_steps, features)로 변경
# X는 [samples, time_steps, features] 형태여야 함
# 각 샘플의 길이를 동일하게 맞추기 위해 padding을 사용할 수도 있습니다.

# X의 shape을 (samples, time_steps, features)로 변환
max_sequence_length = max([len(x) for x in X])  # 가장 긴 시퀀스의 길이

# 시퀀스 길이를 맞추기 위해 padding을 추가
X_padded = np.zeros((len(X), max_sequence_length, X[0].shape[1]))  # 제로 패딩을 추가

for i, seq in enumerate(X):
    X_padded[i, :len(seq), :] = seq

X = X_padded  # 패딩된 X를 최종적으로 사용

# y는 그대로 사용할 수 있습니다.

from sklearn.model_selection import train_test_split

# 7:3 비율로 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 결과 확인
print("훈련 데이터 X:", X_train.shape)
print("테스트 데이터 X:", X_test.shape)
print("훈련 데이터 y:", y_train.shape)
print("테스트 데이터 y:", y_test.shape)


c:\users\songa\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


훈련 데이터 X: (66901, 40, 5)
테스트 데이터 X: (28672, 40, 5)
훈련 데이터 y: (66901,)
테스트 데이터 y: (28672,)


In [25]:
# LSTM 모델 구축
model = Sequential()

# LSTM 레이어 추가: 입력 차원은 X의 특성 개수 (예: anchor_age, gender, icd_version_diag, prev_diag_hashed, icd_code_diag_hashed)
model.add(LSTM(units=64, input_shape=(X.shape[1], X.shape[2]), return_sequences=False))
model.add(Dense(units=32, activation='relu'))  # fully connected layer
model.add(Dense(units=1))  # 예측값은 하나의 연속 값 (icd_code_diag_hashed)

# 모델 컴파일
model.compile(optimizer=Adam(), loss='mean_squared_error')


In [26]:
# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=256)

Epoch 1/10
262/262 [==============================] - 49s 167ms/step - loss: 0.0509
Epoch 2/10
262/262 [==============================] - 43s 165ms/step - loss: 4.7114e-04
Epoch 3/10
262/262 [==============================] - 43s 162ms/step - loss: 3.2185e-04
Epoch 4/10
262/262 [==============================] - 42s 162ms/step - loss: 1.6818e-04
Epoch 5/10
262/262 [==============================] - 43s 163ms/step - loss: 1.5914e-04
Epoch 6/10
262/262 [==============================] - 50s 189ms/step - loss: 1.1498e-04
Epoch 7/10
262/262 [==============================] - 52s 199ms/step - loss: 9.4917e-05
Epoch 8/10
262/262 [==============================] - 49s 188ms/step - loss: 1.0743e-04
Epoch 9/10
262/262 [==============================] - 45s 173ms/step - loss: 1.1490e-04
Epoch 10/10
262/262 [==============================] - 54s 205ms/step - loss: 9.6570e-05


## 모델의 성능 평가

In [27]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# 예측값과 실제 값
y_pred = model.predict(X_test)
y_true = y_test

# MSE, RMSE, MAE, R² 계산
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")


896/896 [==============================] - 24s 26ms/step
MSE: 8.537836271172614e-05
RMSE: 0.009240041272187378
MAE: 0.005990757282875956
R²: 0.9990271821784541


**icd_version_diag와 icd_version 확인(상의 필요)**

In [30]:
# # icd_version_diag와 icd_version 열만 추출해서 새로운 df5 생성
# df5 = df3[['icd_version_diag', 'icd_version']]

# # icd_version이 0인 컬럼 제외(icd_version이 누락된 컬럼 제외)
# df5 = df5[df5['icd_version'] != 0]
# df5.info()

In [31]:
# # 'icd_version_diag'와 'icd_version' 값이 동일한 인스턴스만 필터링
# matching_versions = df5[df5['icd_version_diag'] == df5['icd_version']]

# # 동일한 값의 인스턴스 개수 카운트
# count_matching_versions = matching_versions.shape[0]

# # 결과 출력
# print("Matching instances count:", count_matching_versions)
# print(matching_versions)


In [32]:
# # 'icd_version_diag'와 'icd_version' 값이 다른 인스턴스만 필터링
# different_versions = df5[df5['icd_version_diag'] != df5['icd_version']]

# # 다른 값의 인스턴스 개수 카운트
# count_different_versions = different_versions.shape[0]

# # 결과 출력
# print("Instances with different icd_version_diag and icd_version:", count_different_versions)
# print(different_versions)
